In [1]:
import csv
from decimal import *
from problem3 import Asset, Portfolio

In [2]:
portfolio = Portfolio()

In [3]:
with open('portfolio.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['Symbol']:
            # make an `Asset` and add it to the portfolio
            asset = Asset(**row)
            portfolio.assets.append(asset)

In [4]:
# display portfolio inital state
portfolio.display('T0')


### T0

<Asset(code='BAC', descr='Bank of America Corp', ctry='US', qty=26000, px=11.2400, ccy='USD', val=292240)>
<Asset(code='CSCO', descr='Cisco Systems Inc', ctry='US', qty=100, px=20.8000, ccy='USD', val=2080)>
<Asset(code='F', descr='Ford Motor Co', ctry='US', qty=3100, px=12.6300, ccy='USD', val=39153)>
<Asset(code='FB', descr='Facebook Inc', ctry='US', qty=4300, px=26.8400, ccy='USD', val=115412)>
<Asset(code='GE', descr='General Electric Co', ctry='US', qty=800, px=23.1400, ccy='USD', val=18512)>
<Asset(code='GOOG', descr='Google Inc', ctry='US', qty=1600, px=794.8000, ccy='USD', val=1271680)>
<Asset(code='HPQ', descr='Hewlett-Packard Co', ctry='US', qty=600, px=19.7000, ccy='USD', val=11820)>
<Asset(code='INTC', descr='Intel Corp', ctry='US', qty=300, px=20.8900, ccy='USD', val=6267)>
<Asset(code='JPM', descr='JPMorgan Chase & Co', ctry='US', qty=200, px=48.8000, ccy='USD', val=9760)>
<Asset(code='MRVL', descr='Marvell Technology Group Ltd', ctry='US', qty=900, px=10.3000, c

In [5]:
def get_corporate_actions():
    '''
    iterate through the corportate actions file to create
    a dict of day -> code -> [actions]
    
    the nested dict (although a little more complicated
    means that the asset only needs to be called once 
    per day.
    '''
    corporate_actions = {}
    with open('corporate_actions.csv', 'rb') as f:
        reader = csv.reader(f)
        next(reader, None)  # ignore header row
        for rw in reader:
            day = rw[0]
            code = rw[1]
            if day not in corporate_actions:
                corporate_actions[day] = {}
                corporate_actions[day][code] = [rw[2:]]
            else:
                if code not in corporate_actions[day]:
                    corporate_actions[day][code] = [rw[2:]]
                else:
                    corporate_actions[day][code].append(rw[2:])
                
    return corporate_actions

In [6]:
corporate_actions = get_corporate_actions()

In [7]:
for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
    print '\ncorporate actions: (%s)' % day
    for code in corporate_actions[day]:
        
        asset = portfolio.asset_by_code(code)
        
        for _type, div, ccy in corporate_actions[day][code]:
            print "\t", code, _type, div, ccy
            
            if _type == 'Cash dividend':
                delta_cash = asset.cash_div(float(div))
                portfolio.cash[asset.ccy] += Decimal(delta_cash)

            elif _type == 'Stock dividend':
                asset.stock_div(float(div))

            elif _type == 'Stock split':
                asset.ssplit(div)

            elif _type == 'Name change':
                asset.name_change(div)

            elif _type == 'Symbol change':
                asset.code_change(div)

            else: 
                raise AttributeError("unhandled corporate action type")       
                    
    portfolio.display(day)


corporate actions: (Monday)
	REX Stock split 9:10 
	RIM Symbol change BB 
	RIM Name change Blackberry 
	BAC Cash dividend 0.02 USD

### Monday

<Asset(code='BAC', descr='Bank of America Corp', ctry='US', qty=26000, px=11.2200, ccy='USD', val=291720.0)>
<Asset(code='CSCO', descr='Cisco Systems Inc', ctry='US', qty=100, px=20.8000, ccy='USD', val=2080)>
<Asset(code='F', descr='Ford Motor Co', ctry='US', qty=3100, px=12.6300, ccy='USD', val=39153)>
<Asset(code='FB', descr='Facebook Inc', ctry='US', qty=4300, px=26.8400, ccy='USD', val=115412)>
<Asset(code='GE', descr='General Electric Co', ctry='US', qty=800, px=23.1400, ccy='USD', val=18512)>
<Asset(code='GOOG', descr='Google Inc', ctry='US', qty=1600, px=794.8000, ccy='USD', val=1271680)>
<Asset(code='HPQ', descr='Hewlett-Packard Co', ctry='US', qty=600, px=19.7000, ccy='USD', val=11820)>
<Asset(code='INTC', descr='Intel Corp', ctry='US', qty=300, px=20.8900, ccy='USD', val=6267)>
<Asset(code='JPM', descr='JPMorgan Chase & Co', ctry='U